# Umsetzung mit Gurobi in Python

In [16]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Create a new model
m = gp.Model("Coffee Blending")
m.Params.LogToConsole = 0

# Create variables

super_espresso = m.addVar(name="super_espresso")
deluxe_espresso = m.addVar(name="deluxe_espresso")

# Set objective
m.setObjective(40 * super_espresso + 50 * deluxe_espresso, GRB.MAXIMIZE)

# Add constraints
arabica_constr = m.addConstr(0.5 * super_espresso + 0.25 * deluxe_espresso <= 120, "Arabica")
robusta_constr = m.addConstr(0.5 * super_espresso + 0.75 * deluxe_espresso <= 160, "Robusta")
demand_constr = m.addConstr(deluxe_espresso <= 150, "Nachfrage Deluxe")

# Optimize model
m.optimize()


print(f"Optimal objective value: {m.objVal} EUR")

# Print the solution
for v in m.getVars():
    print(f"{v.varName}, {v.x}")


Optimal objective value: 12000.0 EUR
super_espresso, 200.0
deluxe_espresso, 80.0


## Überprüfung des Modells

Mit komplexen Methoden wird es schwierig, sicherzustellen, dass das Modell korrekt ist. Wir können uns das Modell als eine Textdatei ausgeben lassen und überprüfen, ob es unseren Erwartungen entspricht. Das können wir mit der Methode `write` machen.

In [17]:
m.write("coffee_blending.lp")

with open("coffee_blending.lp", "r") as f:
    print(f.read())

\ Model Coffee Blending
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  40 super_espresso + 50 deluxe_espresso
Subject To
 Arabica: 0.5 super_espresso + 0.25 deluxe_espresso <= 120
 Robusta: 0.5 super_espresso + 0.75 deluxe_espresso <= 160
 Nachfrage_Deluxe: deluxe_espresso <= 150
Bounds
End



## Zugriff auf die Lösung

Nachdem das Modell gelöst wurde, müssen wir auf die Lösung zugreifen, um sie zusammenzufassen und zu interpretieren. In der Regel sind wir an den Werten der Entscheidungsvariablen, dem Wert der Zielfunktion und den Schattenpreisen interessiert.

Die Entscheidungsvariablen haben under anderem die folgenden Attribute:

- `VarName`: Wert der Entscheidungsvariable im Optimum
- `x`: Wert der Entscheidungsvariable im Optimum
- `RC`: Reduzierte Kosten der Entscheidungsvariable
- `SAObjLow`: Sensitivität des Zielfunktionswerts in Bezug auf den Koeffizienten der Entscheidungsvariable in der Zielfunktion
- `SAObjUp`: Sensitivität des Zielfunktionswerts in Bezug auf den Koeffizienten der Entscheidungsvariable in der Zielfunktion
- `SALBLow`: Sensitivität in Bezug auf den unteren Schrankenwert der Entscheidungsvariable
- `SAUBUp`: Sensitivität in Bezug auf den oberen Schrankenwert der Entscheidungsvariable

Die Einschränkungen haben unter anderem die folgenden Attribute:

- `ConstrName`: Name der Einschränkung
- `Slack`: Überschuss der Einschränkung
- `Pi`: Schattenpreis der Einschränkung
- `RHS`: Rechte Seite der Einschränkung
- `SARHSLow`: Sensitivität des Zielfunktionswerts in Bezug auf den rechten Seitenwert der Einschränkung
- `SARHSUp`: Sensitivität des Zielfunktionswerts in Bezug auf den rechten Seitenwert der Einschränkung

Mehr Information über die Attribute des Modellobjekts, Einschränkungen und Variablen finden Sie [hier](https://www.gurobi.com/documentation/11.0/refman/attributes.html#sec:Attributes).


In [3]:
print("Name der Variable super_espresso: ", super_espresso.varName)
print("Wert von Super Espresso im Optimum: ", super_espresso.x)
print("Reduced Cost von Super Espresso: ", super_espresso.RC)
print("Obere Schranke für den Zielfunktionskoeffizienten von Super Espresso: ", super_espresso.SAObjUp)
print("Untere Schranke für den Zielfunktionskoeffizienten von Super Espresso: ", super_espresso.SAObjLow)


Name der Variable super_espresso:  super_espresso
Wert von Super Espresso im Optimum:  200.0
Reduced Cost von Super Espresso:  0.0
Obere Schranke für den Zielfunktionskoeffizienten von Super Espresso:  100.00000000000003
Untere Schranke für den Zielfunktionskoeffizienten von Super Espresso:  33.333333333333336


In [4]:
print("Arabica Constraint Name: ", arabica_constr.constrName)
print("Arabica Constraint Slack: ", arabica_constr.Slack)
print("Arabica Constraint Shadow Price: ", arabica_constr.Pi)
print("Arabica Constraint RHS: ", arabica_constr.RHS)
print("Obere Schranke für die RHS des Arabica Constraints: ", arabica_constr.SARHSUp)
print("Untere Schranke für die RHS des Arabica Constraints: ", arabica_constr.SARHSLow)

Arabica Constraint Name:  Arabica
Arabica Constraint Slack:  0.0
Arabica Constraint Shadow Price:  19.99999999999999
Arabica Constraint RHS:  120.0
Obere Schranke für die RHS des Arabica Constraints:  160.0
Untere Schranke für die RHS des Arabica Constraints:  85.0


Es ist häufig bequemer, die Werte in einen `DataFrame` zu übertragen, um sie zu analysieren.

Das Modellobjekt (in dem Code oben unter `m` gespeichert) hat zwei Methoden, die für uns interessant sind:

- `getVars()`: erlaubt uns über die Entscheidungsvariablen zu iterieren
- `getConstrs()`: erlaubt uns über die Einschränkungen zu iterieren

In [5]:
for v in m.getVars():
    print(v.VarName, v.x, v.RC, v.SAObjLow, v.SAObjUp)

super_espresso 200.0 0.0 33.333333333333336 100.00000000000003
deluxe_espresso 80.0 0.0 19.999999999999993 60.0


In [7]:
for c in m.getConstrs():
    print(c.ConstrName, c.Pi, c.SARHSLow, c.SARHSUp)

Arabica 19.99999999999999 85.0 160.0
Robusta 60.00000000000001 120.0 195.0
Nachfrage Deluxe 0.0 80.0 inf


Es ist häufig bequem, die Lösung in einen `DataFrame` zu übertragen, um sie zu analysieren. Das können wir leicht mit der Bibliothek `pandas` machen.

In [8]:
variables_df = pd.DataFrame(
    [(v.varName, v.x, v.RC) for v in m.getVars()],
    columns=["Name", "Value", "RC"])

variables_df

,Name,Value,RC
0,super_espresso,200.0,0.0
1,deluxe_espresso,80.0,0.0


In [9]:
constr_df = pd.DataFrame(
    [(c.constrName, c.slack, c.Pi, c.SARHSLow, c.SARHSUp) for c in m.getConstrs()],
    columns=["Name", "Slack", "ShadowPrice", "Lower", "Upper"])

print(constr_df)

               Name  Slack  ShadowPrice  Lower  Upper
0           Arabica    0.0         20.0   85.0  160.0
1           Robusta    0.0         60.0  120.0  195.0
2  Nachfrage Deluxe   70.0          0.0   80.0    inf


## Verallgemeinerung und Trennung von Modell und Daten

In der Implementation oben haben wir die Koeffizienten des Modells fest im Code eingegeben. Das funktioniert in dieser kleinen Aufgabe, allerdings ist es nicht flexibel.
Stellen wir uns vor, daß wie den optimalen Plan jeden Monat neu berechnen müssen, weil sich die Preise oder die Nachfrage ändern. In diesem Fall wäre es besser, wenn wir
das Modell und die Koeffizienten trennen. Danach können das Modell mit verschiedenen Koeffizienten laufen lassen. Das ist auch nützlich, wenn wir das Optimum unter verschiedenen Szenarien berechnen wollen (unterschiedliche Preise, Nachfrage, Verfügbarkeit von Ressourcen, etc).


In [10]:
# Define the data

# Define the objective function coefficients
obj_coeffs = [40, 50]

# Define the production constraints
prod_constr_coeff = {
    "arabica": [0.5, 0.25],
    "robusta": [0.5, 0.75],
    "demand": [0, 1],
}

prod_constr_rhs = {
    "arabica": 120,
    "robusta": 160,
    "demand": 150,
}

print(obj_coeffs)
print(prod_constr_rhs)
print(prod_constr_coeff)

[40, 50]
{'arabica': 120, 'robusta': 160, 'demand': 150}
{'arabica': [0.5, 0.25], 'robusta': [0.5, 0.75], 'demand': [0, 1]}


Die `addVars` Methode erlaubt uns, Entscheidungsvariablen zum Modell hinzuzufügen. Es gibt mehrere Möglichkeiten, die Variablen zu definieren (siehe auch die [Dokumentation](https://www.gurobi.com/documentation/11.0/refman/py_model_addvars.html)).

- `addVars(2, name="x")`: fügt zwei Variablen hinzu, die den Namen `x` haben
- `addVars(["a", "b"], name="x")`: fügt zwei Variablen hinzu, die den Namen `x` haben, aber unterschiedliche Indizes haben

In [11]:
# Nützliche Funktionen

print(list(range(10)))

for i in range(10):
    print(i)

print("Über eine Dictionary iterieren\n")

for key, value in prod_constr_coeff.items():
    print("Key=", key, "Value=", value)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1
2
3
4
5
6
7
8
9
Über eine Dictionary iterieren

Key= arabica Value= [0.5, 0.25]
Key= robusta Value= [0.5, 0.75]
Key= demand Value= [0, 1]


In [19]:
mod1 = gp.Model("Coffee Blending")
mod1.Params.LogToConsole = 0

# Create variables

# The data is given as a pandas DataFrame
# With as many columns as there are espresso types
# and one column for the right-hand side
# All constraints are of the form a_1 * x_1 + a_2 * x_2 + ... <= RHS

n_vars = len(obj_coeffs)
espresso = mod1.addVars(n_vars, name="espresso")

# Set objective
mod1.setObjective(gp.quicksum([obj_coeffs[i] * espresso[i] for i in range(n_vars)]), GRB.MAXIMIZE)

# Add constraints
# Wir können über die Dictionary iterieren, um die Nebenbedingungen zu erstellen
for key, value in prod_constr_coeff.items():
    mod1.addConstr(
        gp.quicksum([value[i] * espresso[i] for i in range(n_vars)]) <= prod_constr_rhs[key],
        name=key
    )

mod1.optimize()

print("Optimal objective value:", m.objVal, "EUR")

# Es ist bequem, die Ergebnisse für die Variablen in einem DataFrame zu speichern
# Dasselbe gilt für die Ergebnisse der Nebenbedingungen

vars_df = pd.DataFrame(columns=['Variable', 'Value', 'RC'], data=[(v.VarName, v.X, v.RC) for v in m.getVars()])
constr_df = pd.DataFrame(columns=['Constraint', 'Shadow Price', 'Lower', 'Upper'], data=[(c.ConstrName, c.Pi, c.SARHSLow, c.SARHSUp) for c in m.getConstrs()])

Optimal objective value: 12000.0 EUR


In [20]:
mod1.write("coffee_blending_1.lp")

In [55]:
vars_df

,Variable,Value,RC
0,espresso[0],200.0,0.0
1,espresso[1],80.0,0.0


In [56]:
constr_df

,Constraint,Shadow Price,Lower,Upper
0,arabica,20.0,85.0,160.0
1,robusta,60.0,120.0,195.0
2,demand,0.0,80.0,inf


In [22]:
def build_espresso_blending_model(
        name, 
        obj_coeffs,
        constr_coeffs,
        constr_rhs
    ):
    """
    Builds an espresso blending optimization model using Gurobi.

    Parameters:
    - name (str): The name of the optimization model.
    - obj_coeffs (list): The coefficients of the objective function.
    - constr_coeffs (dict): The constraints for the optimization problem.
    - constr_rhs (dict): The constraints for the optimization problem.

    Returns:
    - m (gurobipy.Model): The built optimization model.
    """

    m = gp.Model(name)
    m.Params.LogToConsole = 0

    # Create variables

    # The data is given as a pandas DataFrame
    # With as many columns as there are espresso types
    # and one column for the right-hand side
    # All constraints are of the form a_1 * x_1 + a_2 * x_2 + ... <= RHS

    n_vars = len(obj_coeffs)
    espresso = m.addVars(n_vars, name="espresso")

    # Set objective
    m.setObjective(gp.quicksum([obj_coeffs[i] * espresso[i] for i in range(n_vars)]), GRB.MAXIMIZE)

    # Add constraints
    # Wir können über die Dictionary iterieren, um die Nebenbedingungen zu erstellen
    for key, value in constr_coeffs.items():
        m.addConstr(
            gp.quicksum([value[i] * espresso[i] for i in range(n_vars)]) <= constr_rhs[key],
            name=key
        )

    return m

In [23]:
m2 = build_espresso_blending_model("Coffee Blending", obj_coeffs, prod_constr_coeff, prod_constr_rhs)
m2.optimize()

# Print the solution
for v in m2.getVars():
    print(f"{v.varName}, {v.x}")

espresso[0], 200.0
espresso[1], 80.0


:::{#exr-gurobi-python-solution-access}

Die Manager des Unternehmens erwägen die Einführung eines neuen Essensangebots (Exotique) und möchten wissen, wie sich das auf den Gewinn auswirken würde. Sie haben die folgenden Informationen:

- Der prognostizierte Gewinn pro kg beträgt 45€
- Der Verbrauch von Robusta beträgt 0.4 kg pro kg Exotique
- Der Verbrauch von Arabica beträgt 0.6 kg pro kg Exotique
- Die maximale Nachfrage nach Exotique beträgt 100 kg

Benutzen Sie die Funktion `build_espresso_blending_model`, um das Optimierungsmodell zu erstellen. Lösen Sie das Modell und beantworten Sie die folgenden Fragen:

1. Wie viele kg Exotique sollten produziert werden?
2. Wie hoch ist der maximale Gewinn, der durch die Einführung von Exotique erzielt werden kann?
3. Wie viel Robusta und Arabica wird für die Produktion von Exotique verwendet?
4. Auf wie viel Euro bewertet das Unternehmen einen Kilogramm Arabica (Schattenpreis)?
6. Lohnt es sich, die Nachfrage nach Exotique zu erhöhen (z.B. durch Werbung)?

- Erstellen Sie einen `DataFrame`, in dem Sie die Werte der Entscheidungsvariablen speichern.
- Erstellen Sie einen `DataFrame`, in dem Sie die Werte der Einschränkungen speichern (Schattenpreise, Überschuss (slack), obere und untere Schranken der rechten Seiten).

:::